In [9]:
#!/usr/bin/env python
# -*- coding: utf-8; py-indent-offset:4 -*-
###############################################################################
#
# Copyright (C) 2018 Daniel Rodriguez
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
#
###############################################################################

In [10]:
import backtrader as bt

In [11]:
#import backtrader.stores.ibstore as ibstore

In [12]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

In [13]:
class St(bt.Strategy): 
    def logdata(self):
        txt = [] 
        txt.append('{}'.format(len(self)))
        txt.append('{}'.format(self.data.datetime.datetime(0).isoformat()))
        txt.append('{:.2f}'.format(self.data.open[0])) 
        txt.append('{:.2f}'.format(self.data.high[0])) 
        txt.append('{:.2f}'.format(self.data.low[0])) 
        txt.append('{:.2f}'.format(self.data.close[0])) 
        txt.append('{:.2f}'.format(self.data.volume[0])) 
        print(','.join(txt))

    data_live = False
    
    def notify_data(self, data, status, *args, **kwargs): 
        print('*' * 5, 'DATA NOTIF:', data._getstatusname(status), *args)
        if status == data.LIVE: 
            self.data_live = True        

#seems this is for buying, selling, make it more structured !
    def notify_order(self, order):
        if order.status == order.Completed:
            buysell = 'BUY ' if order.isbuy() else 'SELL' 
            txt = '{} {}@{}'.format(buysell,order.executed.size,order.executed.price)
            print(txt)           
            
    bought = 0 
    sold = 0
            
    def next(self): 
        self.logdata()
        if not self.data_live: 
            return
        if not self.bought:
            self.bought = len(self) # keep entry bar self.buy()
        elif not self.sold:
            if len(self) == (self.bought + 3):
                self.sell()

In [14]:
def run(args=None):
    cerebro = bt.Cerebro(stdstats=False) 
    store = bt.stores.IBStore(port=7497)

    data = store.getdata(dataname='TSLA', timeframe=bt.TimeFrame.Ticks)
#    data = store.getdata(dataname='TWTR', timeframe=bt.TimeFrame.Ticks)


    cerebro.resampledata(data, timeframe=bt.TimeFrame.Seconds, compression=10) 

    cerebro.broker = store.getbroker()
    
    cerebro.addstrategy(St)

    cerebro.run()

In [15]:
#cerebro.resampledata(data, timeframe=bt.TimeFrame.Seconds,compression=10) 
#cerebro.broker = store.getbroker()
#cerebro.addstrategy(St)

In [16]:
if __name__ == '__main__': 
    run()
    
#Possibly a window in IB Trader goes up, asking if inging connectioins are 
#allowed. Also it work with visual studio    

Server Version: 76
TWS Time at connection:20200617 23:11:17 CET
***** DATA NOTIF: DISCONNECTED
